# Webscraping to gather data from Wikipedia
## Segmenting and Clustering Neighborhoods in Toronto

### _Work in progress_

In [1]:
# Import required libraries for scrapping
import urllib.request
from bs4 import BeautifulSoup

# Import required libraries to handle the data as DataFrame
import pandas as pd

print("Libraries imported!")

Libraries imported!


In [2]:
# set/access url to be scrapped
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)

# parsing through the URL
soup = BeautifulSoup(page, "lxml")

In [3]:
# print(soup.prettify()) # -> uncomment if needed
# soup.table # -> uncommend it needed
all_tables = soup.find_all("table")
right_table = soup.find('table', class_ = 'wikitable sortable')

In [4]:
# Set de Dataframe columns
column_names = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

# Check the Dataframe structure
df

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []

In [5]:
# Looping through the table

for tr_cell in right_table.find_all('tr'):
    row_data = []
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.rstrip())
        
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [6]:
df.head()

PostalCode           Borough               Neighborhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

In [7]:
# Removing records 'Not assigned' for Boroughs
print("Shape prior to the first cleaning:", df.shape)
df = df[df['Borough'] != 'Not assigned']
print("Shape after the first cleaning:", df.shape)

Shape prior to the first cleaning: (180, 3)
Shape after the first cleaning: (103, 3)


In [8]:
df = df.reset_index(drop=True)
df

PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
..         ...               ...   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4          Queen's Park, Ontario Provincial Government  
..                                                 ...  
98       The Kingsway, Montgomery Road, Old Mill North  
99                                Church and Wellesley  
100  Business reply mail Processing Centre, South C...  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  
102  Mimico NW, The Queensway West, South of Bloor,...  

[103 rows x 3 columns]

In [9]:
#df['Neighborhood'].unique()

In [10]:
#df.reset_index(drop=False)
#df = df[['PostalCode', 'Borough', 'Neighborhood']]

In [11]:
#print("Shape before cleaning:",df.shape)
#df = df.set_index('Borough')
#df = df.drop('Not assigned', axis = 0)
#df.reset_index(drop=False, inplace=True)
#print("Shape after cleaning uncategorized Boroughs:",df.shape)

In [12]:
# alternative with Requests
import requests

wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page = requests.get(wiki)

df_raw = pd.read_html(wikipedia_page.content, header=0)[0]
df_new = df_raw[df_raw.Borough != 'Not assigned']

df_new.head()

Postal Code           Borough                                 Neighborhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [13]:
# Check for Neighborhoods which might be 'Not assigned' 
df3 = df_new[df_new['Neighborhood'] == 'Not assigned']

In [14]:
df3

Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []

In [15]:
# shape at the bottom should give [103, 3]